# Clean and Scale Data 

### Contribution and details
- I applyed Jennifer Ngyuen's method of importing data and Adefemi Abimbola's feature extraction and engineering. I ran into problems with accessing information as the way data as accessed and saved differed (JSON -> dataframe vs. csv -> dataframe); it caused some complications when attempting to access nested information like dictionaries. Still attempting to resolve
- I also ran into problems with the size of the data. I did what Jennifer did in her .ipynb, which was use a sample size dataset. Hopefully the techniques are still affective when applied to entire dataset (if that is what we are doing). 
- I added 2 more columns to the dataframe 'master': days_open_count and hours_open_count. They were derived from the column 'business_hours'. I then dropped the column as it is no longer useful.

### Authors: Jennifer Ngyuen, Adefemi Abimbola, Jasmin Pena

In [206]:
# Import libraries and load datasets
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm  # For progress bars
from datetime import datetime # To get hours open
from sklearn.preprocessing import StandardScaler #To scale continous features 

In [207]:
#Paths
business_path = 'yelp_academic_dataset_business.json'
review_path = 'yelp_academic_dataset_review.json'
user_path = 'yelp_academic_dataset_user.json'
checkin_path = 'yelp_academic_dataset_checkin.json'
tip_path = 'yelp_academic_dataset_tip.json'

In [208]:
#Reading JSON files 

def read_json(file_path, max_records=None):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(tqdm(f)):
            if max_records and i >= max_records:
                break
            data.append(json.loads(line))
    return pd.DataFrame(data)
    
# Removed 'max_records' for business and users since we have to clean it
# Saved 'max_records' for others for faster processing but can remove when utilizing entire dataset
business = read_json(business_path)
review = read_json(review_path, max_records=5000)
user = read_json(user_path)
checkin = read_json(checkin_path, max_records=5000)
tip = read_json(tip_path, max_records=5000)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

### Author: Adefemi Abimbola

In [210]:
# Aggregating the dataset into a master dataset
business.rename(columns={'name': 'business_name',
                         'address': 'business_address',
                         'city': 'business_city',
                         'stars': 'business_stars',
                         'review_count':'business_review_count',
                         'attributes':'business_attributes',
                         'categories':'business_categories',
                         'hours':'business_hours'}, inplace=True)
business.drop(columns=['is_open'], inplace=True)
review.rename(columns={'stars': 'review_stars',
                       'date': 'review_date',
                       'text': 'review'}, inplace=True)
review.drop(columns=['useful', 'funny','cool'], inplace=True)
user.rename(columns={'name': 'user_name',
                     'review_count': 'user_review_count',
                     'average_stars': 'user_average_stars',
                     'compliments_total': 'User_compliments_total'}, inplace=True)
user.drop(columns=['elite', 'fans'], inplace=True)
checkin.rename(columns={'date': 'checkin_dates'}, inplace=True)
master = review.merge(business, on='business_id', how='left')
master = master.merge(user, on='user_id', how='left')
master = master.merge(checkin, on='business_id', how='left')
master.dropna(inplace=True)

# master dataset created each row represents one review

In [211]:
#View data 
master.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'review',
       'review_date', 'business_name', 'business_address', 'business_city',
       'state', 'postal_code', 'business_stars', 'business_review_count',
       'business_attributes', 'business_categories', 'business_hours',
       'user_name', 'user_review_count', 'user_average_stars',
       'User_compliments_total', 'checkin_dates'],
      dtype='object')

In [212]:
master.head()

,review_id,user_id,business_id,review_stars,review,review_date,business_name,business_address,business_city,state,...,business_stars,business_review_count,business_attributes,business_categories,business_hours,user_name,user_review_count,user_average_stars,User_compliments_total,checkin_dates
5,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,Dmitri's,795 S 3rd St,Philadelphia,PA,...,4.0,273,"{'BusinessParking': '{'garage': False, 'street...","Mediterranean, Restaurants, Seafood, Greek","{'Wednesday': '17:30-21:0', 'Thursday': '17:30...",Q,4,2.00,1,"2010-03-27 22:04:48, 2010-07-13 00:09:00, 2010..."
32,40thYphUgIfvJq17QCfTwA,QzCEzH3R7Z6erOGLr3t55Q,0pMj5xUAecW9o1P35B0AMw,5.0,Great staff always helps and always nice. Alwa...,2017-05-26 13:10:24,Wawa,2544 W Main Street,Norristown,PA,...,3.5,8,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Food, Coffee & Tea, Gas Stations, Restaurants,...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",Kylhalil,14,4.36,0,"2014-10-01 10:33:34, 2014-10-20 12:23:13, 2014..."
90,byblHsbxiqb1pC1cuSfslA,C_2mNjl-doRVvsL03_T57Q,18eWJFJbXyR9j_5xfcRLYA,4.0,This is the first time I tried this place and ...,2011-10-28 03:43:05,Siam Elephant,509 Linden Ave,Carpinteria,CA,...,4.5,460,"{'RestaurantsGoodForGroups': 'True', 'Alcohol'...","Restaurants, Thai","{'Tuesday': '17:0-21:30', 'Wednesday': '17:0-2...",Richard,28,4.07,2,"2010-03-26 19:31:36, 2010-06-14 03:47:29, 2010..."
125,quiZPC8t-iZs1uiMA1ovEQ,TTibuRAx2gxu-nVAymFijQ,-ikBycdroyTLDBHR9aC3HA,5.0,Stopped in for the lunch menu with my girlfrie...,2014-09-25 18:36:53,Sukho Thai,2450 Music Valley Dr,Nashville,TN,...,3.5,116,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...","Thai, Restaurants","{'Monday': '16:30-21:0', 'Tuesday': '16:30-21:...",Joe,32,4.67,1,"2010-04-22 00:29:12, 2010-11-20 00:03:45, 2010..."
138,zqmkEnp1kfU2vosDcG2kMg,KqKXOl0PMlZGBMlw8OUpyA,-If0ps0QhOLCYVWQWs9RYg,5.0,Yes! I love this place! Maple Street Patisseri...,2013-05-28 21:37:01,Maple Street Patisserie,7638 Maple St,New Orleans,LA,...,3.5,171,"{'Ambience': '{'romantic': False, 'intimate': ...","Restaurants, Food, Sandwiches, Bakeries","{'Tuesday': '6:0-17:0', 'Wednesday': '6:0-17:0...",Jessica,36,4.42,15,"2010-08-02 21:22:49, 2010-08-07 20:57:07, 2010..."


In [213]:
#Learn about dataset
print('Data set info:')
print(master.info())
print()

print('Data set .describe:')
print(master.describe())
print()

print('data set null count:')
print(master.isnull().sum())
print()

Data set info:
<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 5 to 4959
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   review_id               199 non-null    object 
 1   user_id                 199 non-null    object 
 2   business_id             199 non-null    object 
 3   review_stars            199 non-null    float64
 4   review                  199 non-null    object 
 5   review_date             199 non-null    object 
 6   business_name           199 non-null    object 
 7   business_address        199 non-null    object 
 8   business_city           199 non-null    object 
 9   state                   199 non-null    object 
 10  postal_code             199 non-null    object 
 11  business_stars          199 non-null    float64
 12  business_review_count   199 non-null    int64  
 13  business_attributes     199 non-null    object 
 14  business_categories     199 non

In [214]:
#Functions to extract features

def extract_attribute(attr_str, key):
    try:
        attr_dict = ast.literal_eval(attr_str)
        return attr_dict.get(key, None)
    except:
        return None

def extract_parking_type(attr_str, parking_type):
    try:
        attr_dict = ast.literal_eval(attr_str)
        parking_str = attr_dict.get('BusinessParking')
        parking_dict = ast.literal_eval(parking_str) if parking_str else {}
        return parking_dict.get(parking_type, None)
    except:
        return None

def simplify_category(cat_string):
    if not isinstance(cat_string, str):
        return 'Other'
    
    cat_string = cat_string.lower()

    if 'restaurant' in cat_string or 'food' in cat_string:
        return 'Restaurant'
    elif 'shopping' in cat_string or 'store' in cat_string:
        return 'Retail'
    elif 'health' in cat_string or 'medical' in cat_string or 'nutritionist' in cat_string:
        return 'Health'
    elif 'beauty' in cat_string or 'spa' in cat_string or 'salon' in cat_string:
        return 'Beauty'
    elif 'bar' in cat_string or 'nightlife' in cat_string or 'club' in cat_string:
        return 'Nightlife'
    elif 'education' in cat_string or 'school' in cat_string:
        return 'Education'
    elif 'service' in cat_string or 'notary' in cat_string:
        return 'Professional Services'
    elif 'public service' in cat_string or 'government' in cat_string:
        return 'Government'
    elif 'automotive' in cat_string or 'car' in cat_string or 'mechanic' in cat_string or 'tire' in cat_string:
        return 'Automotive'
    elif 'gym' in cat_string or 'fitness' in cat_string or 'yoga' in cat_string or 'active life' in cat_string:
        return 'Fitness'
    elif 'pets' in cat_string or 'animal' in cat_string or 'veterinary' in cat_string:
        return 'Veterinary'
    elif 'hotel' in cat_string or 'lodging' in cat_string or 'travel' in cat_string:
        return 'Travel'
    elif 'church' in cat_string or 'temple' in cat_string or 'synagogue' in cat_string or 'place of worship' in cat_string:
        return 'Religious'
    elif 'movie' in cat_string or 'cinema' in cat_string or 'theater' in cat_string or 'art' in cat_string or 'museum' in cat_string or 'entertainment' in cat_string:
        return 'Entertainment'
    elif 'media' in cat_string or 'news' in cat_string or 'radio' in cat_string or 'television' in cat_string:
        return 'Media/Station'
    else:
        return 'Other'

#Extract number of hours and days open 
def extract_open_info(hours_dict):
    days_open = 0
    hours_open = 0.0
    for day, hours in hours_dict.items():
        if hours != '0:0-0:0' and hours:
            days_open += 1 #Increment day

            #Get hours string
            open_str, close_str = hours.split('-')

            #Convert to datetime 
            open = datetime.strptime(open_str, '%H:%M')
            close = datetime.strptime(close_str, '%H:%M')

            #Get time difference by converting to seconds then convert to hour
            time_diff = (close - open).seconds / 3600

            #Save to hours_open
            hours_open += time_diff

    return pd.Series([days_open, hours_open])

In [215]:
# Preparing Data for Modeling 
master['review_length'] = master['review'].apply(lambda x: len(str(x).split()))

master['Wifi'] = master['business_attributes'].apply(lambda x: extract_attribute(x, 'WiFi'))
master['credit_cards'] = master['business_attributes'].apply(lambda x: extract_attribute(x, 'BusinessAcceptsCreditCards'))
master['bike_parking'] = master['business_attributes'].apply(lambda x: extract_attribute(x, 'BikeParking'))
master['parking_garage'] = master['business_attributes'].apply(lambda x: extract_parking_type(x, 'garage'))
master['parking_lot'] = master['business_attributes'].apply(lambda x: extract_parking_type(x, 'lot'))
master['parking_street'] = master['business_attributes'].apply(lambda x: extract_parking_type(x, 'street'))

# Encode WiFi. 
# *** Wasn't working for me the way @Adefemi had it
master['Wifi'] = master['Wifi'].replace({"'free'": 2, "None": 1, "u'no'": 0, "u'free'": 2, "'no'": 0, "u'paid'": 1, "'paid'": 1, "'None'": 0})
master['Wifi'] = master['Wifi'].fillna(0)

# simplifying the business categories
master['category_simple'] = master['business_categories'].apply(simplify_category)

# Add number of hours and days open
master[['days_open_count', 'hours_open_count']] = master['business_hours'].apply(extract_open_info).apply(pd.Series)

#Drop column business_hours
master = master.drop('business_hours', axis = 1)

/var/folders/zk/15k4kmpn6rx2k9gtwtxw8hnw0000gn/T/ipykernel_6341/4144757867.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  master['Wifi'] = master['Wifi'].fillna(0)


In [216]:
master.head()

,review_id,user_id,business_id,review_stars,review,review_date,business_name,business_address,business_city,state,...,review_length,Wifi,credit_cards,bike_parking,parking_garage,parking_lot,parking_street,category_simple,days_open_count,hours_open_count
5,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,Dmitri's,795 S 3rd St,Philadelphia,PA,...,65,0,None,None,None,None,None,Restaurant,5.0,20.0
32,40thYphUgIfvJq17QCfTwA,QzCEzH3R7Z6erOGLr3t55Q,0pMj5xUAecW9o1P35B0AMw,5.0,Great staff always helps and always nice. Alwa...,2017-05-26 13:10:24,Wawa,2544 W Main Street,Norristown,PA,...,28,0,None,None,None,None,None,Restaurant,0.0,0.0
90,byblHsbxiqb1pC1cuSfslA,C_2mNjl-doRVvsL03_T57Q,18eWJFJbXyR9j_5xfcRLYA,4.0,This is the first time I tried this place and ...,2011-10-28 03:43:05,Siam Elephant,509 Linden Ave,Carpinteria,CA,...,90,0,None,None,None,None,None,Restaurant,6.0,37.0
125,quiZPC8t-iZs1uiMA1ovEQ,TTibuRAx2gxu-nVAymFijQ,-ikBycdroyTLDBHR9aC3HA,5.0,Stopped in for the lunch menu with my girlfrie...,2014-09-25 18:36:53,Sukho Thai,2450 Music Valley Dr,Nashville,TN,...,24,0,None,None,None,None,None,Restaurant,6.0,29.5
138,zqmkEnp1kfU2vosDcG2kMg,KqKXOl0PMlZGBMlw8OUpyA,-If0ps0QhOLCYVWQWs9RYg,5.0,Yes! I love this place! Maple Street Patisseri...,2013-05-28 21:37:01,Maple Street Patisserie,7638 Maple St,New Orleans,LA,...,73,0,None,None,None,None,None,Restaurant,6.0,61.0


In [217]:
master.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'review',
       'review_date', 'business_name', 'business_address', 'business_city',
       'state', 'postal_code', 'business_stars', 'business_review_count',
       'business_attributes', 'business_categories', 'user_name',
       'user_review_count', 'user_average_stars', 'User_compliments_total',
       'checkin_dates', 'review_length', 'Wifi', 'credit_cards',
       'bike_parking', 'parking_garage', 'parking_lot', 'parking_street',
       'category_simple', 'days_open_count', 'hours_open_count'],
      dtype='object')

In [218]:
#Scaling continuous features

#Columns to be scaled
scale_columns = [
    'review_stars',
    'business_review_count',
    'user_review_count',
    'user_average_stars',
    'User_compliments_total',
    'review_length',
    'days_open_count',
    'hours_open_count'
]

#initilize and apply scaler. Chose standard scaler but if another is needed please let me know. 
scaler = StandardScaler()
master[scale_columns] = scaler.fit_transform(master[scale_columns])
master

,review_id,user_id,business_id,review_stars,review,review_date,business_name,business_address,business_city,state,...,review_length,Wifi,credit_cards,bike_parking,parking_garage,parking_lot,parking_street,category_simple,days_open_count,hours_open_count
5,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,-2.058394,I am a long term frequent customer of this est...,2015-09-23 23:10:31,Dmitri's,795 S 3rd St,Philadelphia,PA,...,-0.397852,0,None,None,None,None,None,Restaurant,-0.701262,-1.594000
32,40thYphUgIfvJq17QCfTwA,QzCEzH3R7Z6erOGLr3t55Q,0pMj5xUAecW9o1P35B0AMw,0.883225,Great staff always helps and always nice. Alwa...,2017-05-26 13:10:24,Wawa,2544 W Main Street,Norristown,PA,...,-0.843522,0,None,None,None,None,None,Restaurant,-4.155499,-2.408916
90,byblHsbxiqb1pC1cuSfslA,C_2mNjl-doRVvsL03_T57Q,18eWJFJbXyR9j_5xfcRLYA,0.147820,This is the first time I tried this place and ...,2011-10-28 03:43:05,Siam Elephant,509 Linden Ave,Carpinteria,CA,...,-0.096724,0,None,None,None,None,None,Restaurant,-0.010415,-0.901322
125,quiZPC8t-iZs1uiMA1ovEQ,TTibuRAx2gxu-nVAymFijQ,-ikBycdroyTLDBHR9aC3HA,0.883225,Stopped in for the lunch menu with my girlfrie...,2014-09-25 18:36:53,Sukho Thai,2450 Music Valley Dr,Nashville,TN,...,-0.891702,0,None,None,None,None,None,Restaurant,-0.010415,-1.206915
138,zqmkEnp1kfU2vosDcG2kMg,KqKXOl0PMlZGBMlw8OUpyA,-If0ps0QhOLCYVWQWs9RYg,0.883225,Yes! I love this place! Maple Street Patisseri...,2013-05-28 21:37:01,Maple Street Patisserie,7638 Maple St,New Orleans,LA,...,-0.301491,0,None,None,None,None,None,Restaurant,-0.010415,0.076578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4725,MdGvGVtFZdU-ZLoBgIFo3Q,7ctnUScX9B362qXOebNk9w,-TCa3KBib07_1ko9L2Z0fQ,-1.322989,Foods good but don't order online for delivery...,2018-09-25 15:34:41,Yeoman's Cask and Lion,"200 1st Ave S, Ste B",St. Petersburg,FL,...,-0.963973,0,None,None,None,None,None,Restaurant,0.680433,0.972985
4811,LYBQCkNwLY-Z_2By42vwMg,wcvwEyMLZViMP2rRZAaiZA,1FURjeGJi_LBXcJQg8eskw,0.147820,This place is in a great and fun location and ...,2016-06-23 02:21:01,Padaro Beach Grill,3765 Santa Claus Ln,Carpinteria,CA,...,-0.156950,0,None,None,None,None,None,Restaurant,-0.010415,-0.290135
4920,toLAMehjvvW3JV33WjboRA,1ZhcB8kduDlsC3j70GYAOg,1-z7wd860Rii4kbEMCT8DA,-0.587585,Any breakfast place that I can remember six mo...,2013-10-25 06:49:06,Moon's Kitchen Cafe,712 W Idaho St,Boise,ID,...,-0.301491,0,None,None,None,None,None,Restaurant,-0.701262,-0.697593
4930,r-YKEJT6JjiLIz2YmQ-qNw,uUcNCwf86aea7F9AGFBJNQ,0zf4KKZqbqoxiuKzeZLDdg,-1.322989,I don't know what so special about this place....,2011-01-09 23:01:13,PrimoHoagies,128 S 11th St,Philadelphia,PA,...,-0.879657,0,None,None,None,None,None,Restaurant,0.680433,0.993358
